In [1]:
from bs4 import BeautifulSoup as bs
import re
from dateutil.parser import parse
# from datetime import datetime
import pandas as pd

In [2]:
def parse_html(path): 
    with open(path) as f:
        # read file
        content = f.read()
        # parse with bs
        soup = bs(content, 'lxml')

    return soup

In [3]:
watch_history = parse_html('../Rådata/Takeout/YouTube and YouTube Music/history/watch-history.html')

In [4]:
def is_a_video(tag): 
    '''
    Filter to remove ads and videos that still ex
    takes a block of html code and returns a boolean with whether this is an ad or not
    '''
    not_an_ad = not(bool(re.search('From Google Ads', tag.get_text())))
    not_deleted = len(tag.find_all('a')) > 2
    return (not_an_ad & not_deleted)
       
 

In [5]:
def parse_watch_history(soup):
    '''takes a parsed tree of watch history (wh) from beautiful soup and returns a simple df with the relevant data'''
    # hver yderste blok har klassen 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp' - Her henter vi dem alle sammen
    blocks = soup.find_all(class_ = 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')
    blocks_cleaned = [block for block in blocks if is_a_video(block)]
    print(f'{len(blocks)} entries; {len(blocks_cleaned)} after cleaning. {len(blocks) - len(blocks_cleaned)} ads and deleted videos removed!')
    return blocks_cleaned

watch_history_blocks = parse_watch_history(watch_history)

30345 entries; 28332 after cleaning. 2013 ads and deleted videos removed!


In [6]:
# Finding a video and an add for testing! 
video = watch_history_blocks[0]
ad =  watch_history_blocks[5]

# To create the dataset! 

Det store problem her er at få datoen til at blive hentet rigtigt fra teksten solvom der kan være overlap med titlen af kanalen (hvis den slutter på et tal) eller hvis den har en mærkelig sætning der siger hvornår den er set på - det kan forekomme i nogle tilfælde. 

Den foreløbige løsning er kombinationen af f og r strenge - her kan variable puttes ind i en regex. Det er vigtigt at huske på at kvantifiers i regex '{}' så skal have et dobbelt sæt klammer rund om sig!! 

In [65]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 
watched_at_videos = [video for video in watch_history_blocks if re.search(r'Watched at \d\d:\d\d', video.text)]

In [78]:
watched_at_video = watched_at_videos[0]
watched_at_video.text
match = re.search(r'Watched at \d\d:\d\d', watched_at_video.text)
match.group()

'Watched at 22:16'

In [103]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 

# match = re.search(watch_history_blocks[0].text, r'\\n\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*')
# print(match)
# watch_history_blocks[0].find(class_ = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1').text
block = watched_at_video
print(block.text)
links = block.find_all('a')

channel_title = re.escape(links[1].text)
watched_at = re.search(r'Watched at \d\d:\d\d', block.text) # Watched at 22:16

if watched_at:
   search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
else:
    search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

print(search_string)

match = re.search(search_string, block.text)

print(search_string, match)
match[1]

YouTubeWatched YouTube Music Ableton Walkthroughs: Part 1Red Means RecordingWatched at 22:169 Nov 2019, 23:05:11 CESTProducts: YouTubeWhy is this here? This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  here.
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*)
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*) <re.Match object; span=(57, 117), match='Red Means RecordingWatched at 22:169 Nov 2019, 23>


'9 Nov 2019, 23:05:11 CEST'

In [106]:
video_titles = []
video_links = []
channel_titles = []
channel_links = []
date_watched = []

# i = 0 # debugging! 
for block in watch_history_blocks:
    links = block.find_all('a')
    
    video_titles.append(links[0].text)
    video_links.append(links[0]['href'])
    channel_title = links[1].text # get channel title for date search! 
    channel_titles.append(channel_title)
    channel_links.append(links[1]['href'])

    # Useful for debugging
    # i += 1
    # print(f'{len(links)} links; entry {i} with title {links[0].text}')

    # find out if text has the weird watched at text so we can avoid it! 
    watched_at = re.search(r'Watched at \d\d:\d\d', block.text) 
    channel_title = re.escape(channel_title) # escape so we avoid problems with channel titles full of weird characters! 

    if watched_at:
        search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
    else:
        search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

    date_string = re.search(search_string, block.text)

    if watched_at:
        date_watched.append(parse(date_string.group(1)))
    else:
        date_watched.append(parse(date_string.group(1)))


In [102]:
# date_watched.isoformat()

'2017-08-06T23:44:53+02:00'

In [99]:
date_string = re.search(r'\d?\d \w{3} \d{4}, \d{2}:\d{2}:\d{2}[^P]*', video.text)


In [101]:
date = parse(date_string[0])
date_string[0]

'9 Aug 2022, 13:27:58 CEST'

In [84]:
date.isoformat()

'2022-08-09T13:27:58+02:00'

In [107]:
dataframe = pd.DataFrame({
    'video_title' : video_titles,
    'video_link' : video_links,
    'channel_title' : channel_titles,
    'channel_link' : channel_links,
    'date_watched' : date_watched
})

In [108]:
dataframe.head()

,video_title,video_link,channel_title,channel_link,date_watched
0,Schrödinger's Levels - Astortion Devlog,https://www.youtube.com/watch?v=oPpzT6JKGk0,aarthificial,https://www.youtube.com/channel/UCtEwVJZABCd0t...,2022-08-09 13:27:58+02:00
1,John mayer - neon (after 2 years of practice),https://www.youtube.com/watch?v=dfjTp8YdNgk,אביה מילר,https://www.youtube.com/channel/UC-z1y3pCBSgTT...,2022-08-09 13:27:39+02:00
2,Are Chinese watches worth it?,https://www.youtube.com/watch?v=HljYyn8U4a0,Talking Timepieces,https://www.youtube.com/channel/UCQkmGbeY6QVgw...,2022-08-09 13:13:38+02:00
3,An Intricate $180 Beauty! | The Sugess Moonpha...,https://www.youtube.com/watch?v=O50euB79Jjc,Tone,https://www.youtube.com/channel/UCn0vfLJzUsCFu...,2022-08-09 09:46:47+02:00
4,Corgeut Seamaster 300 Spectre v Omega Seamaste...,https://www.youtube.com/watch?v=93v5Bso2NDU,Wonder of Watches,https://www.youtube.com/channel/UCCPbvLaMx8P-c...,2022-08-09 09:45:38+02:00


In [109]:
dataframe.dtypes

video_title                         object
video_link                          object
channel_title                       object
channel_link                        object
date_watched     datetime64[ns, tzlocal()]
dtype: object